In [1]:
import sys
sys.path.insert(0, "../src")
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from fair import FAIR
from fair.io import read_properties
from fair.interface import fill, initialise
from fair.earth_params import seconds_per_year
import pickle
import xarray as xr
import math
from tabulate import tabulate

In [2]:
ssps = ['ssp126','ssp245','ssp370','ssp585']
climate_models = ['CAMS-CSM1-0_r1i1p1f1', 'GISS-E2-2-G_r1i1p1f1', 'NorESM2-LM_r1i1p1f1',
       'ACCESS-ESM1-5_r1i1p1f1', 'HadGEM3-GC31-LL_r1i1p1f3', 'CESM2_r1i1p1f1',
       'NorCPM1_r1i1p1f1', 'CIESM_r1i1p1f1', 'INM-CM4-8_r1i1p1f1',
       'CNRM-CM6-1-HR_r1i1p1f2', 'CESM2-WACCM_r1i1p1f1', 'CAS-ESM2-0_r1i1p1f1',
       'EC-Earth3-Veg_r1i1p1f1', 'MPI-ESM1-2-HR_r1i1p1f1',
       'MPI-ESM1-2-LR_r1i1p1f1', 'GISS-E2-1-G_r102i1p1f1',
       'GISS-E2-1-G_r1i1p1f1', 'GISS-E2-1-G_r1i1p3f1', 'GISS-E2-1-G_r1i1p5f1',
       'CNRM-CM6-1_r1i1p1f2', 'GISS-E2-1-H_r1i1p1f1', 'GISS-E2-1-H_r1i1p3f1',
       'CMCC-CM2-SR5_r1i1p1f1', 'MRI-ESM2-0_r1i1p1f1', 'MRI-ESM2-0_r4i1p1f1',
       'MRI-ESM2-0_r7i1p1f1', 'MRI-ESM2-0_r13i1p1f1', 'MRI-ESM2-0_r10i1p1f1',
       'MRI-ESM2-0_r1i2p1f1', 'SAM0-UNICON_r1i1p1f1', 'EC-Earth3_r3i1p1f1',
       'EC-Earth3_r8i1p1f1', 'IPSL-CM6A-LR_r1i1p1f1', 'CESM2-FV2_r1i1p1f1',
       'BCC-CSM2-MR_r1i1p1f1', 'ACCESS-CM2_r1i1p1f1', 'MIROC6_r1i1p1f1',
       'FIO-ESM-2-0_r1i1p1f1', 'FIO-ESM-2-0_r2i1p1f1', 'FIO-ESM-2-0_r3i1p1f1',
       'CESM2-WACCM-FV2_r1i1p1f1', 'BCC-ESM1_r1i1p1f1',
       'MPI-ESM-1-2-HAM_r1i1p1f1', 'KACE-1-0-G_r1i1p1f1', 'GFDL-ESM4_r1i1p1f1',
       'UKESM1-0-LL_r1i1p1f2', 'E3SM-1-0_r1i1p1f1', 'KIOST-ESM_r1i1p1f1',
       'IITM-ESM_r1i1p1f1', 'GFDL-CM4_r1i1p1f1', 'CanESM5_r1i1p2f1',
       'CanESM5_r1i1p1f1', 'AWI-CM-1-1-MR_r1i1p1f1', 'INM-CM5-0_r1i1p1f1',
       'FGOALS-f3-L_r1i1p1f1', 'FGOALS-f3-L_r3i1p1f1', 'FGOALS-f3-L_r2i1p1f1',
       'FGOALS-g3_r1i1p1f1', 'CNRM-ESM2-1_r2i1p1f2', 'CNRM-ESM2-1_r3i1p1f2',
       'CNRM-ESM2-1_r1i1p1f2', 'TaiESM1_r1i1p1f1', 'NorESM2-MM_r1i1p1f1',
       'HadGEM3-GC31-MM_r1i1p1f3', 'NESM3_r1i1p1f1', 'MIROC-ES2L_r1i1p1f2']
geos = ['ASIA','REF', 'LAM', 'MAF', 'OECD']
sectors = ['MAGICC AFOLU','MAGICC AFOLU|Agricultural Waste Burning', 'MAGICC AFOLU|Agriculture', 'MAGICC AFOLU|Forest Burning','MAGICC AFOLU|Grassland Burning', 
             'MAGICC AFOLU|Peat Burning', 'MAGICC Fossil and Industrial', 'MAGICC Fossil and Industrial|Aircraft',
            'MAGICC Fossil and Industrial|Energy Sector', 'MAGICC Fossil and Industrial|Industrial Sector',
            'MAGICC Fossil and Industrial|International Shipping','MAGICC Fossil and Industrial|Residential Commercial Other', 
           'MAGICC Fossil and Industrial|Solvents Production and Application','MAGICC Fossil and Industrial|Transportation Sector', 'MAGICC Fossil and Industrial|Waste',]
sectors3 = ['MAGICC AFOLU','MAGICC Fossil and Industrial', 'MAGICC Fossil and Industrial|Aircraft',
            'MAGICC Fossil and Industrial|Energy Sector', 'MAGICC Fossil and Industrial|Industrial Sector',
            'MAGICC Fossil and Industrial|International Shipping','MAGICC Fossil and Industrial|Residential Commercial Other', 
           'MAGICC Fossil and Industrial|Solvents Production and Application','MAGICC Fossil and Industrial|Transportation Sector', 'MAGICC Fossil and Industrial|Waste',]

In [3]:
data2 = {}
sectors2 = [
    '_'.join(word.lower().replace(' ', '_') for word in sector.split('|')) 
    if index in (0, 6) else
    '_'.join(word.lower().replace(' ', '_') for word in sector.split('|') 
             if 'magicc afolu' not in word.lower() and 'magicc fossil and industrial' not in word.lower())
    for index, sector in enumerate(sectors)
]
sectors4 = ['magicc_afolu',
 'magicc_fossil_and_industrial',
 'aircraft',
 'energy_sector',
 'industrial_sector',
 'international_shipping',
 'residential_commercial_other',
 'solvents_production_and_application',
 'transportation_sector',
 'waste']

pickles2 = ['normal_fair']
emission_species2 = ['CH4', 'SULFUR', 'BC', 'OC', 'NH3', 'NOx', 'VOC', 'CO']
emission_species2 = [species.lower() for species in emission_species2]
for specie in emission_species2:
    for sector in sectors2:
        pickles2.append(f'{specie}_no_{sector}_ref1850')
emission_species2.append('n2o')
emission_species2.append('co2')
for specie in emission_species2:
    for geo in geos:
        pickles2.append(f'{specie}_no_{geo}_ref1850')
for sector in sectors4:
    pickles2.append(f'co2_no_{sector}_ref1850')
f_mon_gas = ['CCl4', 'CHCl3', 'CH2Cl2', 'CH3Cl', 'CH3CCl3', 'CH3Br', 'CFC11', 
             'CFC12', 'CFC113', 'CFC114', 'CFC115', 'HCFC22', 'HCFC141b', 
             'HCFC142b', 'Halon1202', 'Halon1211', 'Halon1301', 'Halon2402','CF4', 'C2F6', 'C3F8', 'C4F10', 'C5F12', 
             'C6F14', 'C7F16', 'C8F18', 'NF3', 'SF6', 'SO2F2', 'cC4F8', 'HFC125', 'HFC134a', 'HFC143a', 'HFC152a', 
             'HFC227ea', 'HFC23', 'HFC236fa', 'HFC245fa', 'HFC32', 'HFC365mfc', 'HFC4310mee']
f_mon_gas = [gas.lower() for gas in f_mon_gas]
for gas in f_mon_gas:
    emission_species2.append(gas)
for specie in emission_species2:
    pickles2.append(f'{specie}_no_{specie}_ref1850')
for geo in geos:
    pickles2.append(f'{geo}_no_{geo}_ref1850')
pickles2.append('magicc_afolu_no_magicc_afolu_ref1850')

In [4]:
for pickle_filename in pickles2:
    with open(pickle_filename, 'rb') as file:
        variable_name = pickle_filename.replace('.pkl', '')
        data2[variable_name] = pickle.load(file)
    
    
    for i, ssp in enumerate(ssps[:4]):
        variable_name2 = f'{variable_name}_{ssp}'
        locals()[variable_name2] = data2[variable_name][i]

    for i, ssp in enumerate(ssps[:4]):
        ssp_dataarray2 = locals()[f'{variable_name}_{ssp}']

        for model_index, climate_model in enumerate(climate_models):
            variable_name2 = f'{variable_name}_{ssp}_{model_index + 1}'
            selected_data = ssp_dataarray2.sel(config=climate_model)
            locals()[variable_name2] = selected_data

In [9]:
#normal_fair_ssp126.equals(co2_no_ASIA_ref1850_ssp126)

In [7]:
differences2 = jar_opener(pickles2)

these are equal to normal fair: ['normal_fair', 'ch4_no_aircraft_ref1850', 'ch4_no_solvents_production_and_application_ref1850', 'sulfur_no_agriculture_ref1850', 'sulfur_no_solvents_production_and_application_ref1850', 'bc_no_agriculture_ref1850', 'bc_no_solvents_production_and_application_ref1850', 'oc_no_agriculture_ref1850', 'oc_no_solvents_production_and_application_ref1850', 'nh3_no_solvents_production_and_application_ref1850', 'nox_no_solvents_production_and_application_ref1850', 'voc_no_agriculture_ref1850', 'co_no_agriculture_ref1850', 'co_no_solvents_production_and_application_ref1850', 'co2_no_solvents_production_and_application_ref1850', 'ch3ccl3_no_ch3ccl3_ref1850', 'halon1202_no_halon1202_ref1850', 'magicc_afolu_no_magicc_afolu_ref1850']


In [7]:
#differences2.to_csv('no_specie_differences_ref1850.csv', index=False)

In [6]:
def jar_opener(pickles):
    data = {}
    for pickle_filename in pickles:
        with open(pickle_filename, 'rb') as file:
            variable_name = pickle_filename.replace('.pkl', '')
            data[variable_name] = pickle.load(file)


        for i, ssp in enumerate(ssps[:4]):
            variable_name2 = f'{variable_name}_{ssp}'
            locals()[variable_name2] = data[variable_name][i]

        for i, ssp in enumerate(ssps[:4]):
            ssp_dataarray2 = locals()[f'{variable_name}_{ssp}']

            for model_index, climate_model in enumerate(climate_models):
                variable_name2 = f'{variable_name}_{ssp}_{model_index + 1}'
                selected_data = ssp_dataarray2.sel(config=climate_model)
                locals()[variable_name2] = selected_data
    bad_pickles = []
    for pickle_filename in pickles:
        if (locals()[f'{pickle_filename}_ssp126'].equals(normal_fair_ssp126)):
            bad_pickles.append(pickle_filename)
    print(f'these are equal to normal fair: {bad_pickles}')
#PERCENTAGING  
    table_data = []

    for pickle_filename in pickles:
        row_data = {'pickle_filename': pickle_filename}

        for i in range(4):
            mod = locals()[f'{pickle_filename}_{ssps[i]}'].mean()
            normal = locals()[f'normal_fair_{ssps[i]}'].mean()

            diff_percent = (((locals()[f'{pickle_filename}_{ssps[i]}'] - normal).mean()) / normal.mean()) * 100
            #diff_gmst = mod - normal
            diff_gmst = locals()[f'normal_fair_{ssps[i]}'].mean(axis=1)[-1] - locals()[f'{pickle_filename}_{ssps[i]}'].mean(axis=1)[-1]
            diff_gmst = diff_gmst.item()
            
            
            

            row_data[f'{ssps[i]}%'] = f'{diff_percent:f}%'
            row_data[f'{ssps[i]}_gmst'] = f'{diff_gmst:f}'

        table_data.append(row_data)

    differences = pd.DataFrame(table_data)
